In [1]:
from google.cloud import bigquery
import os

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./maelys-serviceaccount.json"

In [3]:
client = bigquery.Client()

In [4]:
def run_query(bq_client, bq_query):
    query_job = bq_client.query(bq_query)
    return query_job.to_dataframe()

# Check Upload Data Job

In [8]:
QUERY = """
SELECT
*
FROM
  `maelys-interview.maelys.orders6`
LIMIT 1000;
"""

In [9]:
df = run_query(client, QUERY)
df.head(10)

,order_id,user_id,order_created,sku,product_name,quantity,price_per_unit,state
0,22890996,298962,2020-07-01 12:36:00+00:00,7290017683201,1 B-FLAT,1,39.2,Florida
1,22949915,417576,2020-07-01 01:16:00+00:00,7290017683201,1 B-FLAT,1,44.0,North Carolina
2,23348933,427842,2020-07-03 03:19:00+00:00,7290017683201,1 B-FLAT,1,39.2,New York
3,23547656,425359,2020-07-01 01:43:00+00:00,7290017683201,1 B-FLAT,1,39.2,Idaho
4,22898683,276467,2020-07-03 14:51:00+00:00,7290017683201,1 B-FLAT,1,29.0,Connecticut
5,23087747,427727,2020-07-03 00:43:00+00:00,7290017683201,1 B-FLAT,1,29.0,New Jersey
6,23209670,409134,2020-07-04 00:08:00+00:00,7290017683201,1 B-FLAT,1,29.0,New York
7,23362259,427078,2020-07-03 13:31:00+00:00,7290017683201,1 B-FLAT,1,29.0,Texas
8,23388880,207920,2020-07-01 20:33:00+00:00,7290017683201,1 B-FLAT,1,29.0,Michigan
9,23516858,423522,2020-07-04 02:21:00+00:00,7290017683201,1 B-FLAT,1,29.0,California


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   order_id        1000 non-null   int64              
 1   user_id         1000 non-null   int64              
 2   order_created   1000 non-null   datetime64[ns, UTC]
 3   sku             1000 non-null   int64              
 4   product_name    1000 non-null   object             
 5   quantity        1000 non-null   int64              
 6   price_per_unit  1000 non-null   float64            
 7   state           1000 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(4), object(2)
memory usage: 62.6+ KB


# Question 1: Find Total Revenue

In [12]:
QUERY = """
SELECT
    SUM(quantity * price_per_unit) AS TotalRevenue
FROM
  `maelys-interview.maelys.orders6`;
"""

In [13]:
df = run_query(client, QUERY)
df.head()

,TotalRevenue
0,62301.01


#  Question 2: Find Average Order Value (AOV)

In [18]:
QUERY = """
SELECT
    SUM(quantity * price_per_unit)/COUNT(DISTINCT order_id) AS AverageOrderValue, 
    SUM(quantity * price_per_unit) AS TotalRevenue, 
    COUNT(DISTINCT order_id) AS TotalOrders
FROM 
    `maelys-interview.maelys.orders6`;
"""

In [20]:
df = run_query(client, QUERY)
df.head()

,AverageOrderValue,TotalRevenue,TotalOrders
0,89.641741,62301.01,695


#  Question 3: Find LTV (Life-Time Value)


Asumption: For simiplicty 9 days treated as 9 months. the objective is to calculage LTV and not to forecast.

In the given data most of the purchases are one-time purchases and hence the churn is 1, which then just requires us to calculate the ARPU finally, to estimate LTV: we have ARPU and churn, so we just divide one by the other!

ref: https://statsbot.co/blog/calculating-customer-lifetime-value-sql-example/

In [55]:
QUERY = """
WITH
  daily_ARPU AS (
  SELECT
    visit_day,
    AVG(revenue) AS arpu
  FROM (
    SELECT
      user_id,
      DATE_DIFF(DATE(order_created), DATE '2020-07-01', DAY) AS visit_day,
      SUM(quantity * price_per_unit) AS revenue
    FROM
      `maelys-interview.maelys.orders6`
    GROUP BY
      1,
      2)
  GROUP BY
    1)
SELECT
  AVG(arpu) as avg_arpu
FROM
  daily_ARPU;
"""

In [56]:
avg_arpu = run_query(client, QUERY)

Calculating the Churn Rate

In [38]:
QUERY = """
WITH
  daily_visits AS (
  SELECT
    DISTINCT DATE_DIFF(DATE(order_created), DATE '2020-07-01', DAY) AS visit_day,
    user_id
  FROM
    `maelys-interview.maelys.orders6`),
    
    
  c_cal AS (
  SELECT
    past_day.visit_day+1 AS current_day,
    past_day.user_id,
    CASE
      WHEN this_day.user_id IS NULL THEN 'churn'
    ELSE
    'retained'
  END
    AS user_type
  FROM
    daily_visits AS past_day
  LEFT JOIN
    daily_visits AS this_day
  ON
    this_day.user_id=past_day.user_id
    AND this_day.visit_day=past_day.visit_day+1)
    

SELECT
  AVG(churn_rate) AS churn_rate
FROM (
  SELECT
    current_day,
    COUNT(CASE
        WHEN user_type='churn' THEN 1
      ELSE
      NULL
    END
      )/COUNT(user_id) AS churn_rate
  FROM
    c_cal
  GROUP BY
    current_day)
"""

In [51]:
churn_rate = run_query(client, QUERY)

LTV

In [57]:
avg_arpu.squeeze()/churn_rate.squeeze()

86.70919641045637

# Question 4 & Viz 1: Find Revenue by SKU

In [29]:
QUERY = """
SELECT
  sku,
  SUM(quantity * price_per_unit) AS SKURevenue
FROM
  `maelys-interview.maelys.orders6`
GROUP BY
  sku
ORDER BY 2 Desc;
"""

In [30]:
df = run_query(client, QUERY)
df.head(50)

,sku,SKURevenue
0,7290017683379,11569.71
1,7290017683201,9004.90
2,7290017683249,7440.00
3,7290017683003,6073.50
4,7290017683096,4920.00
5,7290017683225,4864.20
6,7290017683263,4395.60
7,7290017683256,3144.75
8,7290017683102,2861.25
9,7290017683218,1591.20


# Viz 2: Show a bar chart of AOV by state:

In [31]:
QUERY = """
SELECT
  state,
  SUM(quantity * price_per_unit)/ COUNT(DISTINCT order_id) AS AverageOrderValue
FROM
  `maelys-interview.maelys.orders6`
GROUP BY
  state
ORDER BY AverageOrderValue desc;
"""

In [32]:
df = run_query(client, QUERY)
df.head(50)

,state,AverageOrderValue
0,Delaware,146.000000
1,Maine,136.975000
2,Kansas,128.730000
3,South Carolina,125.437500
4,Indiana,124.120000
5,Missouri,116.540000
6,Nevada,115.206250
7,West Virginia,112.200000
8,Alaska,112.200000
9,District Of Columbia,112.200000


# Viz 3: Show a line chart of AOV by date

In [28]:
QUERY = """
SELECT
  DATE(order_created) as order_date,
  SUM(quantity * price_per_unit)/COUNT(DISTINCT order_id) AS AverageOrderValue
FROM
  `maelys-interview.maelys.orders6`
GROUP BY
  order_date
ORDER BY order_date asc;
"""

In [27]:
df = run_query(client, QUERY)
df.head(10)

,order_date,AverageOrderValue
0,2020-07-01,90.158861
1,2020-07-02,97.187476
2,2020-07-03,90.293564
3,2020-07-04,85.462069
4,2020-07-05,79.827358
5,2020-07-06,95.947000
6,2020-07-07,89.036364
7,2020-07-08,85.957576
8,2020-07-09,66.512500
